In [130]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.preprocessing.image import img_to_array, load_img

np.random.seed(2018)


In [2]:
def load_numeric_training(standardize=True):
    data = pd.read_csv('./train.csv')
    ID = data.pop('id')
    y = data.pop('species')
    y = LabelEncoder().fit(y).transform(y)
    X = StandardScaler().fit(data).transform(data) if standardize else data.values
    return ID, X, y
load_numeric_training()

(0         1
 1         2
 2         3
 3         5
 4         6
 5         8
 6        10
 7        11
 8        14
 9        15
 10       17
 11       18
 12       20
 13       21
 14       22
 15       25
 16       26
 17       27
 18       29
 19       30
 20       31
 21       32
 22       34
 23       35
 24       37
 25       38
 26       40
 27       42
 28       43
 29       45
        ... 
 960    1541
 961    1543
 962    1544
 963    1545
 964    1547
 965    1548
 966    1549
 967    1550
 968    1551
 969    1552
 970    1554
 971    1555
 972    1556
 973    1557
 974    1559
 975    1561
 976    1562
 977    1563
 978    1566
 979    1568
 980    1569
 981    1570
 982    1571
 983    1572
 984    1574
 985    1575
 986    1578
 987    1581
 988    1582
 989    1584
 Name: id, Length: 990, dtype: int64,
 array([[-0.48661074, -0.13135701, -0.33095592, ..., -0.39487147,
         -0.65214311,  0.26239707],
        [-0.58560191, -0.73488047, -0.02856115, ..., -0.49497401,
 

In [3]:
def load_numeric_test(standardize=True):
    data = pd.read_csv('./test.csv')
    ID = data.pop('id')
    test = StandardScaler().fit(data).transform(data) if standardize else data.values
    return ID, test
load_numeric_test()

(0         4
 1         7
 2         9
 3        12
 4        13
 5        16
 6        19
 7        23
 8        24
 9        28
 10       33
 11       36
 12       39
 13       41
 14       44
 15       46
 16       47
 17       51
 18       52
 19       53
 20       57
 21       59
 22       62
 23       65
 24       68
 25       70
 26       74
 27       77
 28       79
 29       86
        ... 
 564    1493
 565    1495
 566    1497
 567    1498
 568    1503
 569    1510
 570    1513
 571    1517
 572    1522
 573    1526
 574    1528
 575    1533
 576    1534
 577    1535
 578    1537
 579    1540
 580    1542
 581    1546
 582    1553
 583    1558
 584    1560
 585    1564
 586    1565
 587    1567
 588    1573
 589    1576
 590    1577
 591    1579
 592    1580
 593    1583
 Name: id, Length: 594, dtype: int64,
 array([[  1.00641634e-01,  -4.86958483e-01,   1.80047922e+00, ...,
          -5.76080500e-01,  -4.07790442e-01,   1.39997127e+00],
        [ -4.98236705e-01,  -5.889198

In [4]:
def resize_img(img, max_dim=96):
    max_axis = np.argmax(img.size)
    scale = max_dim / img.size[max_axis]
    return img.resize((int(img.size[0] * scale), int(img.size[1] * scale)))


def load_img_data(ids, max_dim=96, center=True):
    X = np.empty((len(ids), max_dim, max_dim, 1))
    for i, id in enumerate(ids):
        img = load_img('images/{}.jpg'.format(id), grayscale=True)
        img = resize_img(img, max_dim=max_dim)
        x = img_to_array(img)
        h, w = x.shape[:2]
        if center:
            h1 = (max_dim - h) >> 1
            h2 = h1 + h
            w1 = (max_dim - w) >> 1
            w2 = w1 + w
        else:
            h1, h2, w1, w2 = 0, h, 0, w
        X[i][h1:h2, w1:w2][:] = x
    return np.around(X / 255)


In [5]:
def load_train_data(split=0.9, random_state=7):
    ID, X_num_train, y = load_numeric_training()
    X_img_train = load_img_data(ID)
    sss = StratifiedShuffleSplit(n_splits=1, train_size=split, test_size=1 - split, random_state=random_state)
    train_idx, val_idx = next(sss.split(X_num_train, y))
    X_num_tr, X_img_tr, y_tr = X_num_train[train_idx], X_img_train[train_idx], y[train_idx]
    X_num_val, X_img_val, y_val = X_num_train[val_idx], X_img_train[val_idx], y[val_idx]
    return (X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val)


def load_test_data():
    ID, X_num_test = load_numeric_test()
    X_img_test = load_img_data(ID)
    return ID, X_num_test, X_img_test


In [6]:
(X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val) = load_train_data()
ID_test, X_num_test, X_img_test = load_test_data()
y_tr, y_val = to_categorical(y_tr), to_categorical((y_val))

In [170]:
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator, array_to_img


class ImageDataGenerator2(ImageDataGenerator):
    def __init__(self, rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                 vertical_flip=True, fill_mode='nearest', X_num_tr=X_num_tr):
        super().__init__(rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                         vertical_flip=True, fill_mode='nearest')
        self.X_num_tr = X_num_tr

    def flow(self, X, y=None, batch_size=32, shuffle=True, seed=None,
             save_to_dir=None, save_prefix='', save_format='jpeg'):
        return NumpyArrayIterator2(X, y, self,
                                   batch_size=batch_size, shuffle=shuffle, seed=seed,
                                   save_to_dir=save_to_dir, save_prefix=save_prefix,
                                   save_format=save_format, X_num_tr=self.X_num_tr)


class NumpyArrayIterator2(NumpyArrayIterator):
    def __init__(self, X, y, generator, batch_size, shuffle, seed,
                 save_to_dir, save_prefix, save_format, X_num_tr):
        super().__init__(X, y, generator, batch_size, shuffle, seed,
                         save_to_dir, save_prefix, save_format)
        self.X_num_tr = X_num_tr

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_X, batch_y = self._get_batches_of_transformed_samples(index_array)
        return {'image': batch_X, 'numerical': self.X_num_tr[index_array]}, batch_y


imgen = ImageDataGenerator2(rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                            vertical_flip=True, fill_mode='nearest', X_num_tr=X_num_tr)
imgen_train = imgen.flow(X_img_tr, y_tr, batch_size=32, seed=7)


In [171]:
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, \
    Flatten, Input, concatenate
from keras.models import Model


def combined_model():
    image = Input(shape=(96, 96, 1), name='image')
    x = Convolution2D(8, (5, 5), input_shape=(96, 96, 1))(image)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    x = Convolution2D(32, (5, 5))(x)
    x = Activation('relu')(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)
    x = Flatten()(x)

    numerical = Input(shape=(192,), name='numerical')
    concatenated = concatenate([x, numerical])

    x = Dense(100, activation='relu')(concatenated)
    x = Dropout(0.5)(x)

    out = Dense(99, activation='softmax')(x)

    model = Model(inputs=[image, numerical], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def image_model():
    image = Input(shape=(96, 96, 1), name='image')
    x = Convolution2D(8, (5, 5), input_shape=(96, 96, 1))(image)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    x = Convolution2D(32, (5, 5))(x)
    x = Activation('relu')(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)
    x = Flatten()(x)

    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)

    out = Dense(99, activation='softmax')(x)

    model = Model(inputs=image, outputs=out)
    model.compile(optimizer=optimizers.Adam(lr=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [175]:
model = combined_model()
model2 = image_model()

In [176]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def combined_generator(imgen):
    while True:
        for i in range(891 // 32):
            batch_x, batch_y = next(imgen)
            while len(imgen.index_array)==0:
                batch_x, batch_y = next(imgen)
            yield batch_x, batch_y


best_model_file = 'leafnet.h5'
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1,
                             save_best_only=True)

history = model.fit_generator(combined_generator(imgen_train),
                              steps_per_epoch=len(X_img_tr) // 32,
                              epochs=99, validation_data=[{'image': X_img_val, 'numerical': X_num_val}, y_val]
                              , verbose=1, callbacks=[best_model])

Epoch 1/99


 1/27 [>.............................] - ETA: 1:03 - loss: 4.6823 - acc: 0.0000e+00

 2/27 [=>............................] - ETA: 32s - loss: 4.6750 - acc: 0.0156     

 3/27 [==>...........................] - ETA: 21s - loss: 4.6600 - acc: 0.0312

 4/27 [===>..........................] - ETA: 16s - loss: 4.6382 - acc: 0.0391

...

23/27 [========================>.....] - ETA: 0s - loss: 0.0344 - acc: 0.9878

24/27 [=========================>....] - ETA: 0s - loss: 0.0371 - acc: 0.9857

25/27 [==========================>...] - ETA: 0s - loss: 0.0364 - acc: 0.9862

26/27 [===========================>..] - ETA: 0s - loss: 0.0353 - acc: 0.9868

Epoch 00099: val_loss improved from 0.00573 to 0.00492, saving model to leafnet.h5
27/27 [==============================] - 4s 137ms/step - loss: 0.0343 - acc: 0.9873 - val_loss: 0.0049 - val_acc: 1.0000


In [108]:
print(X_num_val.shape)
print(X_img_val.shape)
print(y_val.shape)

(99, 192)
(99, 96, 96, 1)
(99, 99)
